In [1]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
%config Completer.use_jedi = False

In [2]:
download_loc = "./index_data/"

if not os.path.exists(download_loc):
    os.mkdir(download_loc)

In [3]:
def website_viewer(main_url, headers_path):

    headers = {
        'authority': 'www.nseindia.com',
        'method': 'GET',
        'path': '',
        'scheme': 'https',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
        'referer': '',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
    }
    
    headers['referer'] = main_url + headers_path
    headers['path'] = headers_path
    
    cookies_url = headers['referer']
    url = headers['referer']
    

    with requests.session() as session:
        # load cookies:
        session.get(cookies_url, headers=headers)

        # get data:
        webpage = requests.get(url, headers=headers)
        
    return webpage

In [4]:
main_url = "https://www.nseindia.com"
headers_path = "/products-services/indices-broad-market"

# visiting the webpage
webpage = website_viewer(main_url, headers_path)

if webpage.status_code != 200:
    print("ERROR . . .")

In [7]:
webpage.content

b'<!DOCTYPE html>\n<html lang="en">\n\n<head>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1">\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\n<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" />\n<title>Broad Market Indices - NSE India</title>\n<meta name="description" content="Broad Market Indices: These indices are broad-market indices, consisting of the large, liquid stocks listed on the Exchange. They serve as a benchmark for measuring the performance of the stocks or portfolios.\r\n" />\n<meta name="keywords" content="" />\n<meta name="robots" content="index,follow" />\n<link rel="shortcut icon" href="/assets/images/favicon.ico" type="image/x-icon" />\n<meta property="og:title" content="Broad Market Indices - NSE India" />\n<meta property="og:description" content="Broad Market Indices: These indices are broa

In [8]:
# reading webpage
page_source = BeautifulSoup(webpage.content)

In [9]:
page_source

<!DOCTYPE html>
<html lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"/>
<title>Broad Market Indices - NSE India</title>
<meta content="Broad Market Indices: These indices are broad-market indices, consisting of the large, liquid stocks listed on the Exchange. They serve as a benchmark for measuring the performance of the stocks or portfolios.
" name="description"/>
<meta content="" name="keywords"/>
<meta content="index,follow" name="robots"/>
<link href="/assets/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<meta content="Broad Market Indices - NSE India" property="og:title"/>
<meta content="Broad Market Indices: These indices are broad-market indices, consisting of the large, liquid stocks liste

In [10]:
# Extracting URLs indices URLs from page_source
indices_urls_info = page_source.find_all('h2', class_="section-heading withborder")[0].findAllNext('ul')[0].findAll('a')
indices_url = {}
for i,index in enumerate(indices_urls_info):
    print("{0:3}. {1:30} : {2:20}".format(i+1, index.text, main_url+index['href']))
    
    # saving indices name as key and url as value in dictionary
    indices_url[index.text] = index['href']

  1. NIFTY 50 Index                 : https://www.nseindia.com/products-services/indices-nifty50-index
  2. NIFTY Next 50 Index            : https://www.nseindia.com/products-services/indices-niftynext50-index
  3. NIFTY 100 Index                : https://www.nseindia.com/products-services/indices-nifty100-index
  4. NIFTY 200 Index                : https://www.nseindia.com/products-services/indices-nifty200-index
  5. NIFTY 500 Index                : https://www.nseindia.com/products-services/indices-nifty500-index
  6. NIFTY Midcap150 Index          : https://www.nseindia.com/products-services/indices-niftymidcap150-index
  7. NIFTY Midcap 50 Index          : https://www.nseindia.com/products-services/indices-niftymidcap50-index
  8. NIFTY Midcap 100 Index         : https://www.nseindia.com/products-services/indices-niftymidcap100-index
  9. NIFTY Smallcap 250 Index       : https://www.nseindia.com/products-services/indices-niftysmallcap250-index
 10. NIFTY Smallcap 50 Index        :

In [16]:
for i,index_ in enumerate(indices_url.items()):
    print("{0:3}. {1:30} : {2:20}".format(i+1, index_[0], main_url+index_[1]))

    # visiting the webpage
    webpage = website_viewer(main_url, index_[1])

    # reading webpage
    index_page = BeautifulSoup(webpage.content)

    # Extracting all files url present on the page
    files_url = index_page.findAll('a')

    for file_url in files_url:
        if file_url.text.find("Download List of") != -1:
            print(f"\t{file_url.text} :: {file_url['href']}")

            df = pd.read_csv(file_url['href'])
            print("\tShape of data :: ", df.shape, end='\n\n')
            df.to_csv(os.path.join(download_loc,file_url['href'].split('/')[-1]),index=False)
    break

  1. NIFTY 50 Index                 : https://www.nseindia.com/products-services/indices-nifty50-index
	Download List of NIFTY 50 stocks (.csv) :: https://archives.nseindia.com/content/indices/ind_nifty50list.csv
	Shape of data ::  (50, 5)



In [17]:
# Combining all company details
company_details = {}
for file in os.listdir(download_loc):
    df = pd.read_csv(os.path.join(download_loc,file))
    for details in df.values:
        isin_code = details[-1]
        if isin_code not in company_details:
            company_details[isin_code] = details
            
company_details_df = pd.DataFrame(data=company_details.values(),columns=df.columns.values)
company_details_df.to_csv("./company_details.csv",index=False)
company_details_df

,Company Name,Industry,Symbol,Series,ISIN Code
0,Adani Ports and Special Economic Zone Ltd.,SERVICES,ADANIPORTS,EQ,INE742F01042
1,Asian Paints Ltd.,CONSUMER GOODS,ASIANPAINT,EQ,INE021A01026
2,Axis Bank Ltd.,FINANCIAL SERVICES,AXISBANK,EQ,INE238A01034
3,Bajaj Auto Ltd.,AUTOMOBILE,BAJAJ-AUTO,EQ,INE917I01010
4,Bajaj Finance Ltd.,FINANCIAL SERVICES,BAJFINANCE,EQ,INE296A01024
5,Bajaj Finserv Ltd.,FINANCIAL SERVICES,BAJAJFINSV,EQ,INE918I01018
6,Bharat Petroleum Corporation Ltd.,OIL & GAS,BPCL,EQ,INE029A01011
7,Bharti Airtel Ltd.,TELECOM,BHARTIARTL,EQ,INE397D01024
8,Britannia Industries Ltd.,CONSUMER GOODS,BRITANNIA,EQ,INE216A01030
9,Cipla Ltd.,PHARMA,CIPLA,EQ,INE059A01026
